# KV Cache

- inference 중에 Key(K), Value(V) 값을 재사용(reuse)할 수 있도록, 중간의 K,V 계산값을 저장해 응답의 생성 속도를 향상시키는 기법.
- 코드가 조금 복잡해지고, 메모리 사용량이 증가하고, training 중에는 사용할 수 없다는 것이 단점.
- 하지만 이런 단점에도 불구하고 LLM을 배포할 때 inference의 speed-up은 그만한 가치가 있음.

## How it works?

- LLM이 텍스트를 생성하는 과정을 생각해보자.
- 예를들어, "Time flies"라는 prompt가 주어진 상황을 보자.

![image](https://sebastianraschka.com/images/LLMs-from-scratch-images/bonus/kv-cache/kv-cache-attn-1.png)

- 이전 챕터(2, 4)에서 다뤘던 것 처럼, LLM은 한번에 하나의 token을 생성함.
  - "fast"라는 단어를 생성했다면 다음으로 주어지는 prompt는 "Time flies fast"가 된다.

![image](https://sebastianraschka.com/images/LLMs-from-scratch-images/bonus/kv-cache/kv-cache-attn-2.png)

- 두 그림을 비교해서 보면, 처음 두 token의 K, V vector는 동일하므로, 다음 token을 생성할때마다 이를 다시 계산하는 것은 상당히 비효율 적.
- 따라서, KV-cache의 아이디어는 **이전에 생성했던 Key-Value 값을 따로 저장**해두었다가 이를 재사용(reuse)하는 caching mechanism을 구현하는 것.
  - 불필요한(중복되는) 계산을 방지하는데 많은 도움이 된다.

## Implementation

- 구현법에는 여러 방법이 있지만, key idea는 각 생성 step에서 새롭게 생성된 token에 대해서만 Key-Value tensor를 계산하는 것.

1. Register cache buffer
   - `MultiHeadAttention`에서, `cache_k`와 `cache_v`라는 2개의 buffer를 추가.

2. Forward pass with `use_cache` flag
   - `MultiHeadAttention`의 `forward()`에 인자로 `use_cache`를 추가.
   - 새로운 token을 `keys_new`, `values_new` 및 `queries`에 project한 후, key-value 값을 초기화하거나 cache에 추가

3. Clear the cache
   - text 생성 시, 독립적인 시퀀스 사이(e.g. text generation call 사이)에 2개의 buffer를 모두 재설정 해야 하므로, `MultiHeadAttention` class에서 cache를 reset하는 method를 추가.

4. Propagate `use_cache` in the full model
   - `MultiHeadAttention` class를 수정했으므로, `GPTModel` class도 수정해야 함.
   - 현재 token index의 위치를 추적하는 instructor를 추가하고
   - block을 호출하는 1-line을 loop로 대체하고, 각 transformer block을 통해 `use_cache`를 전달
     - 마찬가지로 `TransformerBlock` class가 `use_cache`를 인자로 받도록 수정
   - `GPTModel`에서 model-level에서의 reset 기능을 추가, 모든 block의 cache를 한번에 지울 수 있도록 함.

5. Using the cache in generation
   - 앞선 수정사항들을 반영한 text generation function을 작성

In [2]:
import tiktoken
import time
import torch
import torch.nn as nn

In [ ]:
# 03_multihead_attention.py에서 작성했던 MultiHeadAttention 사용
class MultiHeadAttention(nn.Module):
    def __init__(self, d_in, d_out, context_length, dropout, num_heads, qkv_bias=False):
        super().__init__()
        assert d_out % num_heads == 0, "d_out must be divisible by num_heads"
        
        self.d_out = d_out
        self.num_heads = num_heads
        self.d_head = d_out // num_heads

        self.W_query = nn.Linear(d_in, d_out, bias=qkv_bias)    # Query weight
        self.W_key = nn.Linear(d_in, d_out, bias=qkv_bias)      # Key weight
        self.W_value = nn.Linear(d_in, d_out, bias=qkv_bias)    # Value weight
        self.out_projection = nn.Linear(d_out, d_out)           # Last output projection(head의 ouput들을 concat한 결과물)
        self.dropout = nn.Dropout(dropout)

        # module을 GPU로 보낼 때, mask도 함께 GPU로 이동.
        self.register_buffer("mask", torch.triu(torch.ones(context_length, context_length), diagonal=1))

        ####################################
        # NEW
        self.register_buffer("cache_k", None, persistent=False)  # persistent=False로 설정하여 state_dict에 저장되지 않도록 함
        self.register_buffer("cache_v", None, persistent=False)
        self.ptr_current_pos = 0
        ####################################

    def forward(self, x, use_cache=False):
        b, num_tokens, d_in = x.shape   # batch 단위로 처리하므로 batch dimension인 B가 추가

        # input projection (Q, K, V 생성)
            # [B, n_tokens, d_in] -> [B, n_tokens, d_out]
        Q, K_new, V_new = self.W_query(x), self.W_key(x), self.W_value(x)

        # split 함수 사용(head 단위로 분할)
            # [B, n_tokens, d_out] -> [B, num_heads, n_tokens, d_head]
        Q, K_new, V_new = self.split(Q), self.split(K_new), self.split(V_new)  
        ####################################
        # NEW
        if use_cache:
            if self.cache_k is None:
                # cache가 비어있는 경우, 현재 K, V를 그대로 캐시에 저장
                self.cache_k, self.cache_v = K_new, V_new
            else:
                # cache에 이전 K, V가 있는 경우, 현재 K, V를 이어붙임
                self.cache_k = torch.cat([self.cache_k, K_new], dim=2)  # n_tokens 차원(2번째 차원)을 따라 이어붙임
                self.cache_v = torch.cat([self.cache_v, V_new], dim=2)
            K, V = self.cache_k, self.cache_v
        else:
            K, V = K_new, V_new
        ####################################

        # self attention 연산
            # Q * K^T -> [B, num_heads, n_tokens, n_tokens]
        attn_scores = Q @ K.transpose(-2, -1)


        ####################################
        # NEW
        num_tokens_Q = Q.shape[-2]
        num_tokens_K = K.shape[-2]
        if use_cache:
            mask_bool = self.mask.bool()[
                self.ptr_current_pos : self.ptr_current_pos + num_tokens_Q, :num_tokens_K
            ]
            self.ptr_current_pos += num_tokens_Q
        else:
            mask_bool = self.mask.bool()[:num_tokens_Q, :num_tokens_K]

        ####################################

        # masking 처리 -> future token을 보지 못하도록.
        attn_scores.masked_fill_(
            mask_bool, -torch.inf
        )

        # softmax scaling 및 dropout
        attn_weights = torch.softmax(attn_scores / K.shape[-2]**0.5, dim=-1)
        attn_weights = self.dropout(attn_weights)

        # (Q * K^T) * V 연산
            # [B, num_heads, n_tokens, d_head]
        context_vector = attn_weights @ V

        # concat 함수 사용, head 단위로 분할된 context_vector를 다시 concat
            # [B, num_heads, n_tokens, d_head] -> [B, n_tokens, d_out]
        context_vector = self.concat(context_vector)  
        context_vector = self.out_projection(context_vector)  # output projection

        return context_vector

    def split(self, tensor):
        """
        split tensor by number of heads

        Input shape: [B, n_tokens, d_out]
        Output shape: [B, num_heads, n_tokens, d_tensor]
        """

        b, n_tokens, d_out = tensor.shape

        d_tensor = d_out // self.num_heads
        tensor = tensor.view(b, n_tokens, self.num_heads, d_tensor).transpose(1, 2)

        return tensor
    
    def concat(self, tensor):
        """
        concat tensor by number of heads

        Input shape: [B, num_heads, n_tokens, d_tensor]
        Output shape: [B, n_tokens, d_out]
        """

        b, num_heads, n_tokens, d_tensor = tensor.size()

        d_out = num_heads * d_tensor
        tensor = tensor.transpose(1, 2).contiguous().view(b, n_tokens, d_out)

        return tensor
    
    ####################################
    # NEW
    def reset_cache(self):
        self.cache_k, self.cache_v, = None, None
        self.ptr_current_pos = 0
    ####################################

In [5]:
from _04_gpt import LayerNorm, FeedForward, generate_text_simple

class TransformerBlock(nn.Module):
    def __init__(self, cfg):
        super().__init__()

        self.attention = MultiHeadAttention(
            d_in = cfg['embed_dim'],
            d_out = cfg['embed_dim'],
            context_length = cfg['context_length'],
            num_heads = cfg['num_heads'],
            dropout = cfg['drop_rate'],
            qkv_bias = cfg['qkv_bias']
        )

        self.ffn = FeedForward(cfg)

        self.norm1 = LayerNorm(cfg['embed_dim'])
        self.norm2 = LayerNorm(cfg['embed_dim'])

        self.drop_shortcut = nn.Dropout(cfg['drop_rate'])

    def forward(self, x, use_cache=False):
        """
        input -> LayerNorm -> MHA -> Dropout -> skip connection
        -> LayerNorm -> FFN -> Dropout -> skip connection
        -> output
        """
        # attention with skip connection
        residual = x
        x = self.norm1(x)           # LayerNorm

        ####################################
        # NEW
        x = self.attention(x, use_cache=use_cache)       # MHA, [batch, context_length, embed_dim]
        ####################################

        x = self.drop_shortcut(x)   # Dropout
        x = x + residual            # skip(residual) connection

        # FFN with skip connection
        residual = x
        x = self.norm2(x)           # LayerNorm
        x = self.ffn(x)             # FeedForward
        x = self.drop_shortcut(x)   # Dropout
        x = x + residual            # skip(residual) connection

        return x

In [7]:
class GPTModel(nn.Module):
    def __init__(self, cfg):
        super().__init__()

        self.token_embedding = nn.Embedding(cfg['vocab_size'], cfg['embed_dim'])
        self.position_embedding = nn.Embedding(cfg['context_length'], cfg['embed_dim'])
        self.drop_embedding = nn.Dropout(cfg['drop_rate'])

        ####################################
        # NEW
        self.transformer_blocks = nn.ModuleList(
            [TransformerBlock(cfg) for _ in range(cfg['num_layers'])]
        )
        self.current_pos = 0
        ####################################

        self.final_norm = LayerNorm(cfg['embed_dim'])
        self.out_head = nn.Linear(cfg['embed_dim'], cfg['vocab_size'], bias=False)

    def forward(self, in_idx, use_cache=False):
        batch_size, seq_length = in_idx.shape

        # token embedding, positional embedding을 더해서 최종 input embedding 구성
        token_embeddings = self.token_embedding(in_idx)
        
        ####################################
        # NEW
        if use_cache:
            pos_ids = torch.arange(
                self.current_pos, self.current_pos + seq_length, device=in_idx.device, dtype=torch.long
            )
            self.current_pos += seq_length
        else:
            pos_ids = torch.arange(
                0, seq_length, device=in_idx.device, dtype=torch.long
            )
        pos_embeddings = self.position_embedding(pos_ids).unsqueeze(0)
        ####################################

        x = token_embeddings + pos_embeddings   # [batch_size, num_tokens, embed_dim]

        x = self.drop_embedding(x)

        # Transformer block forward pass
        ####################################
        # NEW
        for block in self.transformer_blocks:
            x = block(x, use_cache=use_cache)
        ####################################

        # last layer norm
        x = self.final_norm(x)

        logits = self.out_head(x)

        return logits
    
    ####################################
    # NEW
    def reset_kv_cache(self):
        for block in self.transformer_blocks:
            block.attention.reset_cache()
        self.current_pos = 0

In [8]:
def generate_text_simple_cached(model, idx, max_new_tokens, context_size=None, use_cache=True):
    model.eval()
    context_length = context_size or model.position_embedding.num_embeddings

    with torch.no_grad():
        if use_cache:
            # Initialize cache with full prompt
            model.reset_kv_cache()
            logits = model(idx[:, -context_length:], use_cache=True)

            for _ in range(max_new_tokens):
                # 가장 높은 log-probability를 가진 token 선택 (greedy sampling)
                next_idx = logits[:, -1].argmax(dim=-1, keepdim=True)

                # 새로운 token을 입력 sequence에 추가
                idx = torch.cat([idx, next_idx], dim=1)

                # model에는 새 token만을 전달
                logits = model(next_idx, use_cache=True)
        
        else:
            for _ in range(max_new_tokens):
                logits = model(idx[:, -context_length:], use_cache=False)
                next_idx = logits[:, -1].argmax(dim=-1, keepdim=True)
                idx = torch.cat([idx, next_idx], dim=1)

    return idx           

In [9]:
GPT_CONFIG_124M = {
        'vocab_size': 50257,        # Vocabulary size
        'context_length': 1024,     # Context(max sequence) length
        'embed_dim': 768,           # Embedding dimension
        'num_heads': 12,            # Number of attention heads
        'num_layers': 12,           # Number of layers(transformer blocks)
        'drop_rate': 0.1,           # Dropout rate
        'qkv_bias': False,          # Q,K,V bias
    }

In [16]:
torch.manual_seed(62)

model = GPTModel(GPT_CONFIG_124M)

model = GPTModel(GPT_CONFIG_124M)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print(f"Using device: {device}")

model.to(device)
model.eval()  # disable dropout

start_context = "O say can you see,"

tokenizer = tiktoken.get_encoding("gpt2")
encoded = tokenizer.encode(start_context)
encoded_tensor = torch.tensor(encoded, device=device).unsqueeze(0)

print(f"\n{50*'='}\n{22*' '}IN\n{50*'='}")
print("\nInput text:", start_context)
print("Encoded input text:", encoded)
print("encoded_tensor.shape:", encoded_tensor.shape)
    
if torch.cuda.is_available():
    torch.cuda.synchronize()
start = time.time()

token_ids = generate_text_simple_cached(
    model = model,
    idx = encoded_tensor,
    max_new_tokens = 200
)

if torch.cuda.is_available():
    torch.cuda.synchronize()
total_time = time.time() - start

decoded_text = tokenizer.decode(token_ids.squeeze(0).tolist())
print(f"\n\n{50*'='}\n{22*' '}OUT\n{50*'='}")
print("\nOutput:", token_ids)
print("Output length:", len(token_ids[0]))
print("Output text:", decoded_text)

print(f"\nTime: {total_time:.2f} sec")
print(f"{int(len(token_ids[0])/total_time)} tokens/sec")

if torch.cuda.is_available():
    max_mem_bytes = torch.cuda.max_memory_allocated()
    max_mem_gb = max_mem_bytes / (1024 ** 3)
    print(f"Max memory allocated: {max_mem_gb:.2f} GB")

Using device: cuda

                      IN

Input text: O say can you see,
Encoded input text: [46, 910, 460, 345, 766, 11]
encoded_tensor.shape: torch.Size([1, 6])


                      OUT

Output: tensor([[   46,   910,   460,   345,   766,    11,  8138, 35660, 48808, 30016,
          6786,  7302,  8059, 15650,  1548, 42937, 17486, 16213, 21924, 17527,
         32383, 42783, 27128,  4918, 47812,  5965, 31670, 49894, 32969, 14771,
         34579, 36355, 17382, 25496, 12783, 24405, 20614, 40610, 13629, 22232,
         46780, 41907,  1209, 15390,  9182, 49822,  4442, 36900, 44489, 29012,
         12648, 17555, 10780, 32523, 38994, 44818, 37698, 40017, 10933, 15858,
         29446, 41775, 46986, 31764,  1877, 11512,  8884, 34415, 12440, 26747,
         21238, 12476, 20876, 20191, 31772, 35027, 19674, 24399, 19200, 34365,
          6591, 37650, 24198,  2730, 20479, 24609, 28307, 22033,  5524, 35576,
          8430, 41268, 33943, 42958, 48436, 43138,  5779, 23894, 17574, 33282,
      

In [15]:
from _04_gpt import main

main()

Using device: cuda

                      IN

Input text: Hello, I am
Encoded input text: [15496, 11, 314, 716]
encoded_tensor.shape: torch.Size([1, 4])


                      OUT

Output: tensor([[15496,    11,   314,   716, 38718, 11139,  4535, 46798, 39622, 20124,
          9799,  1330, 13403, 14447,  4748, 30387, 43330, 19030, 33606,  9908,
         18323, 29347, 36465, 46135,  3177, 41268, 49233,  5027, 44525, 28370,
         18210, 38214, 33964, 29150, 38376, 23223, 34025, 33589, 10189, 26951,
          1404, 10914,  5383, 17720, 43639,  5607, 46954, 15773, 29286, 13582,
         44134, 12551, 38363,  9995, 12420,  4536, 19795, 33606, 35105, 32756,
         41971, 48907, 22706, 32522, 32289, 24683, 22067, 29461, 13228,  8932,
          5536,  6072, 14258, 39508, 32757, 10243, 13411, 40950, 13869, 19253,
          7202, 35644,  2297,  4533,  5171, 39923,  4785,  2770,  4442, 24681,
         26380, 26702,  6744, 24616, 42528, 48837,  5998, 25043, 11646,  5684,
         29422, 4402

- KV-cache를 사용한 코드는 초당 59 token을 처리하고, 사용하지 않은 pure GPTModel은 초당 41 token을 처리함.
  - 또한 KV-cache를 사용한 코드는 실행에 3.47s가 걸렸고, 사용하지 않은 코드는 4.96s가 걸림.


- 이처럼, 시퀀스 길이가 증가함에 따라 KV-cache의 장점과 단점은 다음과 같이 더욱 두드리점.
  - 장점: **계산 효율성이 좋아짐** (Computational efficiency increases)
    - caching이 없다면, step t에서의 attention은 새로운 Q를 이전 t개의 K와 모두 비교해야 하므로, 누적되는 작업량은 제곱에 비례해서 증가 $\rightarrow$ $O(n^2)$
    - 하지만 caching을 사용하면, 각 K와 V가 한번만 계산된 후 재사용 되므로 전체 step당 복잡도가 선형인 $O(n)$이 됨.
  - 단점: **메모리 사용량 증가** (Memory usage increases linearly)
    - 새로운 token이 추가될 때 마다 KV-cache에 저장되므로, 긴 sequence와 large LLM의 경우 누적되는 KV-cache의 크기가 커져서 상당한 양의 GPU 메모리를 사용하게 됨.
    - 임시 방편으로 KV-cache의 크기를 줄일 수 있지만, 이렇게 되면 복잡성이 더욱 증가하게 됨. (하지만 LLM을 deploy 할때는 그럴만한 가치가 있다고 함.)


[KV-cache 설명해주는 youtube 영상](https://youtu.be/80bIUggRJf4)